In [8]:
from preprocess import *
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, LSTM, Activation
from keras.utils import to_categorical
import wandb
from wandb.keras import WandbCallback
import matplotlib.pyplot as plt
import sklearn.metrics as metrics

In [9]:
wandb.init()
config = wandb.config

config.max_len = 21
config.buckets = 50

# Save data to array file first
save_data_to_array(max_len=config.max_len, n_mfcc=config.buckets)

#labels=np.array(["chirping_birds", "crickets", "crow", 
#                 "frog", "insects"])
labels=np.array(["BRA", "BAM", "BBI", 
                 "BMA", "BIN"])

Saving vectors of label - 'BRA': 100%|████████████████████████████████████████████████| 12/12 [00:00<00:00, 120.30it/s]


In [10]:
# Loading train/test set
X_train, X_test, X_val, y_train, y_test, y_val = get_train_test()

In [11]:
# Setting channels to 1 to generalize stereo sound to 1 channel
channels = 1
config.epochs = 50
config.batch_size = 100

# Number of classes
num_classes = 5

# Reshape X_train and X_test to include a 4th dimension (channels)
X_train = X_train.reshape(X_train.shape[0], config.buckets, config.max_len, channels)
X_test = X_test.reshape(X_test.shape[0], config.buckets, config.max_len, channels)
X_val = X_val.reshape(X_val.shape[0], config.buckets, config.max_len, channels)

In [12]:
# Spectrogram visualized of 0th element
print(X_train.shape)
#plt.imshow(X_train[500, :, :, 0])

(649, 50, 21, 1)


In [13]:
# Getting vector number where each number corresponds to a label
y_train_hot = to_categorical(y_train)
y_test_hot = to_categorical(y_test)
y_val_hot = to_categorical(y_val)

In [14]:
# Building the model
model = Sequential()

input_shape= (config.buckets, config.max_len, channels)

model.add(Conv2D(24, (3, 3), strides=(1, 1), input_shape=input_shape))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (3, 3), padding="valid"))
model.add(MaxPooling2D((2, 2), strides=(2, 2)))
model.add(Activation('relu'))

model.add(Conv2D(48, (3, 1), padding="valid"))
model.add(Activation('relu'))

model.add(Flatten())
model.add(Dropout(rate=0.5))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(rate=0.5))

model.add(Dense(len(labels)))
model.add(Activation('softmax'))
model.summary()
# Conv2D: 
#    Filters: 32
#    Kernel_size: (3,3) (height/width of the 2D convolution window)     
'''model.add(Conv2D(32, (3, 3),
    input_shape=(config.buckets, config.max_len, channels),
    activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))'''

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 48, 19, 24)        240       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 24, 9, 24)         0         
_________________________________________________________________
activation_5 (Activation)    (None, 24, 9, 24)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 22, 7, 48)         10416     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 11, 3, 48)         0         
_________________________________________________________________
activation_6 (Activation)    (None, 11, 3, 48)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 9, 3, 48)         

"model.add(Conv2D(32, (3, 3),\n    input_shape=(config.buckets, config.max_len, channels),\n    activation='relu'))\n\nmodel.add(MaxPooling2D(pool_size=(2, 2)))\n\nmodel.add(Flatten())\n\nmodel.add(Dense(128, activation='relu'))\nmodel.add(Dense(num_classes, activation='softmax'))"

In [15]:
# Configure CNN for training
model.compile(loss="categorical_crossentropy",
                  optimizer="adam",
                  metrics=['accuracy'])

In [16]:
wandb.init()
print(y_train_hot.shape)
print(labels.shape)
print(X_train.shape)
# Train the CNN model
#    X_train: Input data
#    y_train_hot: Target data
model.fit(X_train, y_train_hot, epochs=config.epochs, validation_data=(X_val, y_val_hot), callbacks=[WandbCallback(data_type="image", labels=labels)])

(649, 5)
(5,)
(649, 50, 21, 1)

Train on 649 samples, validate on 434 samples
Epoch 1/50
649/649 [==============================] - ETA: 18s - loss: 4.5785 - accuracy: 0.250 - ETA: 9s - loss: 3.7529 - accuracy: 0.328 - ETA: 6s - loss: 3.4751 - accuracy: 0.43 - ETA: 4s - loss: 3.2928 - accuracy: 0.46 - ETA: 3s - loss: 2.9507 - accuracy: 0.52 - ETA: 3s - loss: 2.6885 - accuracy: 0.55 - ETA: 2s - loss: 2.4328 - accuracy: 0.58 - ETA: 2s - loss: 2.3398 - accuracy: 0.58 - ETA: 1s - loss: 2.1312 - accuracy: 0.58 - ETA: 1s - loss: 1.9995 - accuracy: 0.57 - ETA: 0s - loss: 1.9268 - accuracy: 0.57 - ETA: 0s - loss: 1.7802 - accuracy: 0.60 - ETA: 0s - loss: 1.6815 - accuracy: 0.60 - ETA: 0s - loss: 1.6417 - accuracy: 0.60 - ETA: 0s - loss: 1.6113 - accuracy: 0.60 - 2s 3ms/step - loss: 1.5630 - accuracy: 0.6225 - val_loss: 0.7253 - val_accuracy: 0.7350
Epoch 2/50
649/649 [==============================] - ETA: 0s - loss: 0.8659 - accuracy: 0.65 - ETA: 1s - loss: 0.8175 - accuracy: 0.70 - ETA: 0s -

Epoch 11/50
649/649 [==============================] - ETA: 0s - loss: 0.6877 - accuracy: 0.68 - ETA: 0s - loss: 0.5325 - accuracy: 0.78 - ETA: 0s - loss: 0.5262 - accuracy: 0.78 - ETA: 0s - loss: 0.5029 - accuracy: 0.80 - ETA: 0s - loss: 0.4684 - accuracy: 0.81 - ETA: 0s - loss: 0.4503 - accuracy: 0.82 - ETA: 0s - loss: 0.4202 - accuracy: 0.84 - ETA: 0s - loss: 0.4071 - accuracy: 0.84 - ETA: 0s - loss: 0.4123 - accuracy: 0.84 - ETA: 0s - loss: 0.4001 - accuracy: 0.84 - ETA: 0s - loss: 0.3979 - accuracy: 0.85 - ETA: 0s - loss: 0.4200 - accuracy: 0.84 - ETA: 0s - loss: 0.4283 - accuracy: 0.84 - ETA: 0s - loss: 0.4254 - accuracy: 0.84 - 1s 2ms/step - loss: 0.4225 - accuracy: 0.8444 - val_loss: 0.4427 - val_accuracy: 0.8525
Epoch 12/50
649/649 [==============================] - ETA: 0s - loss: 0.1251 - accuracy: 0.96 - ETA: 1s - loss: 0.2022 - accuracy: 0.96 - ETA: 1s - loss: 0.3082 - accuracy: 0.93 - ETA: 1s - loss: 0.2834 - accuracy: 0.92 - ETA: 0s - loss: 0.3418 - accuracy: 0.89 - ETA:

649/649 [==============================] - ETA: 0s - loss: 0.2512 - accuracy: 0.87 - ETA: 0s - loss: 0.3177 - accuracy: 0.84 - ETA: 0s - loss: 0.2751 - accuracy: 0.86 - ETA: 0s - loss: 0.2815 - accuracy: 0.86 - ETA: 0s - loss: 0.2594 - accuracy: 0.87 - ETA: 0s - loss: 0.2449 - accuracy: 0.88 - ETA: 0s - loss: 0.2290 - accuracy: 0.89 - ETA: 0s - loss: 0.2240 - accuracy: 0.90 - ETA: 0s - loss: 0.2231 - accuracy: 0.90 - ETA: 0s - loss: 0.2160 - accuracy: 0.90 - ETA: 0s - loss: 0.2141 - accuracy: 0.90 - ETA: 0s - loss: 0.2290 - accuracy: 0.90 - ETA: 0s - loss: 0.2305 - accuracy: 0.90 - ETA: 0s - loss: 0.2472 - accuracy: 0.89 - ETA: 0s - loss: 0.2530 - accuracy: 0.89 - ETA: 0s - loss: 0.2447 - accuracy: 0.89 - ETA: 0s - loss: 0.2490 - accuracy: 0.89 - 1s 2ms/step - loss: 0.2482 - accuracy: 0.8983 - val_loss: 0.5055 - val_accuracy: 0.8825
Epoch 22/50
649/649 [==============================] - ETA: 1s - loss: 0.1187 - accuracy: 0.96 - ETA: 1s - loss: 0.2384 - accuracy: 0.90 - ETA: 1s - loss: 

649/649 [==============================] - ETA: 1s - loss: 0.2651 - accuracy: 0.90 - ETA: 0s - loss: 0.2476 - accuracy: 0.90 - ETA: 1s - loss: 0.2378 - accuracy: 0.90 - ETA: 0s - loss: 0.2800 - accuracy: 0.88 - ETA: 0s - loss: 0.2622 - accuracy: 0.89 - ETA: 0s - loss: 0.2510 - accuracy: 0.90 - ETA: 0s - loss: 0.2515 - accuracy: 0.89 - ETA: 0s - loss: 0.2257 - accuracy: 0.90 - ETA: 0s - loss: 0.2191 - accuracy: 0.90 - ETA: 0s - loss: 0.2355 - accuracy: 0.90 - ETA: 0s - loss: 0.2290 - accuracy: 0.90 - ETA: 0s - loss: 0.2122 - accuracy: 0.90 - ETA: 0s - loss: 0.2070 - accuracy: 0.91 - ETA: 0s - loss: 0.2013 - accuracy: 0.91 - 1s 2ms/step - loss: 0.1992 - accuracy: 0.9153 - val_loss: 0.5899 - val_accuracy: 0.8641
Epoch 32/50
649/649 [==============================] - ETA: 0s - loss: 0.1728 - accuracy: 0.93 - ETA: 0s - loss: 0.1653 - accuracy: 0.93 - ETA: 0s - loss: 0.1273 - accuracy: 0.95 - ETA: 0s - loss: 0.0969 - accuracy: 0.97 - ETA: 0s - loss: 0.0865 - accuracy: 0.96 - ETA: 0s - loss: 

649/649 [==============================] - ETA: 1s - loss: 0.0726 - accuracy: 1.00 - ETA: 0s - loss: 0.1056 - accuracy: 0.96 - ETA: 0s - loss: 0.0980 - accuracy: 0.96 - ETA: 1s - loss: 0.1043 - accuracy: 0.95 - ETA: 0s - loss: 0.1051 - accuracy: 0.95 - ETA: 0s - loss: 0.1105 - accuracy: 0.95 - ETA: 0s - loss: 0.1095 - accuracy: 0.95 - ETA: 0s - loss: 0.1195 - accuracy: 0.94 - ETA: 0s - loss: 0.1348 - accuracy: 0.94 - ETA: 0s - loss: 0.1450 - accuracy: 0.94 - ETA: 0s - loss: 0.1402 - accuracy: 0.94 - ETA: 0s - loss: 0.1422 - accuracy: 0.94 - ETA: 0s - loss: 0.1377 - accuracy: 0.94 - 1s 2ms/step - loss: 0.1318 - accuracy: 0.9492 - val_loss: 0.6164 - val_accuracy: 0.8779
Epoch 42/50
649/649 [==============================] - ETA: 0s - loss: 0.0898 - accuracy: 0.96 - ETA: 0s - loss: 0.0564 - accuracy: 0.98 - ETA: 0s - loss: 0.0657 - accuracy: 0.98 - ETA: 0s - loss: 0.0611 - accuracy: 0.98 - ETA: 0s - loss: 0.0531 - accuracy: 0.98 - ETA: 0s - loss: 0.0992 - accuracy: 0.96 - ETA: 0s - loss: 

In [17]:
# Save the keras model
model.save("bio_cnn_model.h5")
print("Model has been saved.")

Model has been saved.


## Running the IntelliChirp Biophony CNN

In [18]:
from keras.models import load_model

# Load the model
loaded_model = load_model('ant_cnn_model.h5')

In [19]:
# Summarize the model
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 48, 19, 24)        240       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 24, 9, 24)         0         
_________________________________________________________________
activation_5 (Activation)    (None, 24, 9, 24)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 22, 7, 48)         10416     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 11, 3, 48)         0         
_________________________________________________________________
activation_6 (Activation)    (None, 11, 3, 48)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 9, 3, 48)         

In [20]:
y_pred_ohe = loaded_model.predict(X_test)  # shape=(n_samples, 12)
y_pred_labels = np.argmax(y_pred_ohe, axis=1)  # only necessary if output has one-hot-encoding, shape=(n_samples)

confusion_matrix = metrics.confusion_matrix(y_true=y_test, y_pred=y_pred_labels)  # shape
print(confusion_matrix)

for class_i in range(len(labels)) :
    indices = np.argwhere(y_test == class_i)
    sum = 0
    for index in indices:
        sum += (y_test[index] == y_pred_labels[index])
    if(len(indices) > 0) : mean = sum/len(indices)
    else : mean = "N/A"
    print("Accuracy for class", labels[class_i], ":", mean)

print("Overall Accuracy :", np.mean(y_test == y_pred_labels))

[[ 28   4   0   0   0]
 [  0 202   5   0   0]
 [  0  10  13   0   0]
 [  0   0   1   0   0]
 [  0   7   1   0   0]]
Accuracy for class BRA : [0.875]
Accuracy for class BAM : [0.97584541]
Accuracy for class BBI : [0.56521739]
Accuracy for class BMA : [0.]
Accuracy for class BIN : [0.]
Overall Accuracy : 0.8966789667896679


In [21]:
## Running the model

n_mfcc = config.buckets
max_len = config.max_len
# convert file to wav2mfcc
# Mel-frequency cepstral coefficients
file_path = "./prediction/nature_sc.wav"
big_wave, sr = librosa.load(file_path, mono=True, sr=None)
#print(wave.shape, sr)

classification = []

for sec_index in range( int(big_wave.shape[0] / sr) ) :
    start_sec = sec_index
    end_sec = sec_index + 1
    
    sec_to_trim = np.array( [ float(start_sec), float(end_sec) ] )
    print(sec_to_trim)
    sec_to_trim = np.ceil( sec_to_trim * sr )

    wave = big_wave[int(sec_to_trim[0]) : int(sec_to_trim[1])]
    print(wave)

    wave = np.asfortranarray(wave[::3])
    mfcc = librosa.feature.mfcc(wave, sr=16000, n_mfcc=n_mfcc)

    # If maximum length exceeds mfcc lengths then pad the remaining ones
    if (max_len > mfcc.shape[1]):
        pad_width = max_len - mfcc.shape[1]
        mfcc = np.pad(mfcc, pad_width=((0, 0), (0, pad_width)), mode='constant')

    # Else cutoff the remaining parts
    else:
        mfcc = mfcc[:, :max_len]

    # Convert wav to MFCC
    prediction_data = wav2mfcc('./prediction/nature_sc.wav')
    prediction_data = mfcc
    print(prediction_data.shape)
    #print(wav2mfcc())
    # Reshape to 4 dimensions
    prediction_data = prediction_data.reshape(1, config.buckets, config.max_len, channels)
    #prediction_data = prediction_data.reshape(1, 20, config.max_len, channels)

    # Run the model on the inputted file
    predicted = loaded_model.predict(prediction_data)

    # Output the prediction values for each class
    print ('PREDICTED VALUES')
    labels_indices = range(len(labels))
    max_value = 0
    max_value_index = 0
    for index in labels_indices:
        print('\n', labels[index], ": ", '%.08f' % predicted[0,index])
        if predicted[0,index] > max_value:
            max_value_index = index
            max_value = predicted[0,index]

    # Output the prediction
    if max_value < 0.5:
        print("GUESS: Nothing")
        classification.append( { "class" : "Nothing", "timestamp" : start_sec } )
    else:
        print('\n\nGUESS: ', labels[max_value_index])
        classification.append( { "class" : labels[max_value_index], "timestamp" : start_sec } )

print(classification)

[0. 1.]
[ 0.0000000e+00  1.5258789e-05  0.0000000e+00 ...  3.3020020e-02
  1.2680054e-02 -8.7432861e-03]
(50, 21)
PREDICTED VALUES

 BRA :  0.00000002

 BAM :  0.99999845

 BBI :  0.00000156

 BMA :  0.00000001

 BIN :  0.00000000


GUESS:  BAM
[1. 2.]
[-0.03717041 -0.05769348 -0.06455994 ...  0.01766968  0.01895142
  0.01779175]
(50, 21)
PREDICTED VALUES

 BRA :  0.00000011

 BAM :  0.65600646

 BBI :  0.34398925

 BMA :  0.00000400

 BIN :  0.00000018


GUESS:  BAM
[2. 3.]
[ 0.02345276  0.02101135  0.01712036 ... -0.01161194 -0.0141449
 -0.01431274]
(50, 21)
PREDICTED VALUES

 BRA :  0.00000021

 BAM :  0.99949670

 BBI :  0.00050283

 BMA :  0.00000016

 BIN :  0.00000001


GUESS:  BAM
[3. 4.]
[-0.01583862 -0.01066589 -0.00762939 ... -0.0377655  -0.03556824
 -0.02685547]
(50, 21)
PREDICTED VALUES

 BRA :  0.00034440

 BAM :  0.99944335

 BBI :  0.00012089

 BMA :  0.00007492

 BIN :  0.00001626


GUESS:  BAM
[4. 5.]
[-0.02836609 -0.02510071 -0.02012634 ...  0.0138855  -0.00386047
 -

 BIN :  0.00004231


GUESS:  BAM
[{'class': 'BAM', 'timestamp': 0}, {'class': 'BAM', 'timestamp': 1}, {'class': 'BAM', 'timestamp': 2}, {'class': 'BAM', 'timestamp': 3}, {'class': 'BAM', 'timestamp': 4}, {'class': 'BAM', 'timestamp': 5}, {'class': 'BAM', 'timestamp': 6}, {'class': 'BAM', 'timestamp': 7}, {'class': 'BAM', 'timestamp': 8}, {'class': 'BAM', 'timestamp': 9}, {'class': 'BAM', 'timestamp': 10}, {'class': 'BAM', 'timestamp': 11}, {'class': 'BAM', 'timestamp': 12}, {'class': 'BAM', 'timestamp': 13}, {'class': 'BAM', 'timestamp': 14}, {'class': 'BAM', 'timestamp': 15}, {'class': 'BAM', 'timestamp': 16}, {'class': 'BAM', 'timestamp': 17}, {'class': 'BAM', 'timestamp': 18}, {'class': 'BAM', 'timestamp': 19}, {'class': 'BAM', 'timestamp': 20}, {'class': 'BAM', 'timestamp': 21}, {'class': 'BAM', 'timestamp': 22}, {'class': 'BAM', 'timestamp': 23}, {'class': 'BAM', 'timestamp': 24}, {'class': 'BAM', 'timestamp': 25}, {'class': 'BAM', 'timestamp': 26}, {'class': 'BAM', 'timestamp': 2